In [1]:
import numpy as np
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget
import os
import pandas as pd
import pickle
from tqdm import tqdm
from scipy.fft import fft, fftfreq
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score


In [2]:
#read data
create_data_pickle = False
if create_data_pickle:
    files = os.listdir('data/train')
    list_of_dicts = []
    types_to_include = ['ax', 'ay', 'az', 'phone_ax', 'phone_ay', 'phone_az', 'speed', 'longitude', 'latitude', 'altitude', 'phone_steps']

    for file in tqdm(files):
        Dict = {}
        d = Recording(os.path.join('data/train',file))

        Dict['labels'] = d.labels
        for data_type in types_to_include:
            if data_type in d.data.keys():
                Dict[data_type] = d.data[data_type]
        list_of_dicts.append(Dict)

    data = pd.DataFrame(list_of_dicts)
    data.to_pickle(path='data/pickled_and_sorted_training_data.pkl.zst', compression={'method': 'zstd'})
else:
    data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')


In [3]:
#load pickled training 3d norm accelerometer data
import pickle
import pandas as pd

file = open('accel_mag_train.pkl', 'rb')
pickled = pickle.load(file)
accel_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(accel_mag_train.shape)
file.close()

labels = []
for label in data["labels"]:
    labels.extend([[label["path_idx"],label["watch_loc"]]])
labels = pd.DataFrame(labels, columns =['path_idx', "smartwatch_location"])
labels



(396, 1)


,path_idx,smartwatch_location
0,2,1
1,0,0
2,0,0
3,2,1
4,1,1
...,...,...
391,3,2
392,4,0
393,2,1
394,3,0


In [21]:
altitude_data = [x.values[::15] for x in data["altitude"]]

In [22]:
dataframe_altitude = pd.DataFrame(altitude_data)

In [10]:
def features_extraction(df): 
    
    FEATURES = ['MIN','MAX','MEAN','RMS','VAR','STD','POWER','PEAK','P2P','CREST FACTOR','SKEW','KURTOSIS',
            'MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f']
    
    Min=[];Max=[];Mean=[];Rms=[];Var=[];Std=[];Power=[];Peak=[];Skew=[];Kurtosis=[];P2p=[];CrestFactor=[];
    FormFactor=[]; PulseIndicator=[];
    Max_f=[];Sum_f=[];Mean_f=[];Var_f=[];Peak_f=[];Skew_f=[];Kurtosis_f=[]
    
    X = df.values
    ## TIME DOMAIN ##
    #list of lists of lists, ugly as fuck but it works 
    for recording in X:
        Min.append(np.min(recording[0]))
        Max.append(np.max(recording[0]))
        Mean.append(np.mean(recording[0]))
        Rms.append(np.sqrt(np.mean(recording[0]**2)))
        Var.append(np.var(recording[0]))
        Std.append(np.std(recording[0]))
        Power.append(np.mean(recording[0]**2))
        Peak.append(np.max(np.abs(recording[0])))
        P2p.append(np.ptp(recording[0]))
        CrestFactor.append(np.max(np.abs(recording[0]))/np.sqrt(np.mean(recording[0]**2)))
        Skew.append(stats.skew(recording[0]))
        Kurtosis.append(stats.kurtosis(recording[0]))
        FormFactor.append(np.sqrt(np.mean(recording[0]**2))/np.mean(recording[0]))
        PulseIndicator.append(np.max(np.abs(recording[0]))/np.mean(recording[0]))
        ## FREQ DOMAIN ##
        ft = fft(recording[0])
        S = np.abs(ft**2)/len(df)
        Max_f.append(np.max(S))
        Sum_f.append(np.sum(S))
        Mean_f.append(np.mean(S))
        Var_f.append(np.var(S))
        
        Peak_f.append(np.max(np.abs(S)))
        Skew_f.append(stats.skew(recording[0]))
        Kurtosis_f.append(stats.kurtosis(recording[0]))

    
    #Create dataframe from features
    df_features = pd.DataFrame(index = [FEATURES], 
                               data = [Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,
                                       Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f])
    return df_features

In [11]:
#extract features from accelerometer norm and get rid of multiindex
features = pd.DataFrame.transpose(features_extraction(accel_mag_train))
features.columns = features.columns.map(''.join)
features

,MIN,MAX,MEAN,RMS,VAR,STD,POWER,PEAK,P2P,CREST FACTOR,SKEW,KURTOSIS,MAX_f,SUM_f,MEAN_f,VAR_f,PEAK_f,SKEW_f,KURTOSIS_f
0,0.153406,2.597480,1.202418,1.221131,0.045352,0.212961,1.491161,2.597480,2.444074,2.127110,0.176443,0.543339,5.469889e+07,5.641470e+07,460.904406,2.444401e+10,5.469889e+07,0.176443,0.543339
1,0.046882,2.826338,1.092686,1.149698,0.127844,0.357553,1.321806,2.826338,2.779456,2.458331,0.346138,-0.193552,4.173153e+07,4.619995e+07,392.696408,1.480367e+10,4.173153e+07,0.346138,-0.193552
2,0.083755,3.057903,1.093724,1.135764,0.093728,0.306151,1.289960,3.057903,2.974148,2.692375,1.456056,3.479315,4.175404e+07,4.502559e+07,382.974880,1.482894e+10,4.175404e+07,1.456056,3.479315
3,0.276498,2.088655,1.026057,1.083954,0.122162,0.349516,1.174955,2.088655,1.812157,1.926886,0.348455,-0.611317,3.086272e+07,3.444390e+07,319.682780,8.841351e+09,3.086272e+07,0.348455,-0.611317
4,0.080601,3.464102,1.078819,1.119106,0.088548,0.297570,1.252399,3.464102,3.383500,3.095418,2.066954,9.037060,1.734226e+07,1.866169e+07,242.940102,3.915198e+09,1.734226e+07,2.066954,9.037060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,0.093611,3.464102,1.551828,1.678641,0.409665,0.640051,2.817836,3.464102,3.370491,2.063635,0.761946,-0.359174,8.212287e+07,9.609317e+07,826.906681,5.804152e+10,8.212287e+07,0.761946,-0.359174
392,0.025811,3.464102,1.468600,1.625606,0.485809,0.697000,2.642595,3.464102,3.438291,2.130960,0.905103,-0.054596,7.030573e+07,8.614188e+07,758.184439,4.351259e+10,7.030573e+07,0.905103,-0.054596
393,0.034486,3.464102,1.294628,1.425602,0.356279,0.596891,2.032342,3.464102,3.429616,2.429922,1.162461,0.824875,1.761473e+07,2.135908e+07,331.086919,4.811941e+09,1.761473e+07,1.162461,0.824875
394,0.032321,3.464066,0.996864,1.078101,0.168563,0.410565,1.162301,3.464066,3.431746,3.213119,0.854506,1.884827,4.478016e+07,5.237601e+07,392.082963,1.501348e+10,4.478016e+07,0.854506,1.884827


In [25]:
from scipy.signal import find_peaks

altitude_gradients = []
for sample in altitude_data:
    altitude_gradients.append(np.gradient(sample))

In [26]:
#### Try to make slopes based on not every signal but on every 20th signal or so

In [27]:
altitude_gradients = pd.DataFrame(altitude_gradients)

In [50]:
altitude_data[0][-1]

452.98732080776244

In [55]:
first = []
for i in range(len(altitude_data)):
    first.append(altitude_data[i][0])

last = []
for i in range(len(altitude_data)):
    last.append(altitude_data[i][-1])

In [56]:
pd.DataFrame(first)

,0
0,407.332413
1,407.676311
2,456.399994
3,471.688363
4,457.399994
...,...
391,451.697559
392,436.882629
393,456.395513
394,501.199982


In [57]:
total = pd.concat([features,altitude_gradients,pd.DataFrame(first),pd.DataFrame(last)],axis=1)

In [60]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

def split_and_train(X_train,y_train):

    X_train.columns = [''] * len(X_train.columns)
    X_train.fillna(0, inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(
        X_train.to_numpy(), y_train.to_numpy(), test_size=0.1, random_state=10)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train) 
    X_test = scaler.fit_transform(X_test) 

    param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

    # Instantiate model with 1000 decision trees, use all cores
    rf = RandomForestClassifier()

    # Use random search to find the best hyperparameters
    rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=10, 
                                 cv=5)

    # Fit the random search object to the data
    rand_search.fit(X_train, y_train)

    y_pred = np.round(rand_search.predict(X_test),0)

    print(y_pred)
    return rand_search,y_pred,y_test,X_train,X_test

rf_model,y_pred,y_test,X_train,X_test = split_and_train(total,labels["path_idx"])

[4 2 2 2 3 4 4 2 0 2 4 3 3 4 4 1 2 4 2 4 2 4 3 3 2 2 3 4 2 3 2 2 4 1 3 2 2
 2 4 2]


In [61]:
print(accuracy_score(y_pred,y_test))

0.6
